#### Function calling
- OpenAI 모델이 사용자 코드 또는 외부 서비스와 상호 작용할 수 있도록 지원하는 기능
- LLM 모델은 데이터베이스 조회, 서버 APU 호출, 파일저장 등 이런 작업 하는 것이 불가하기에 실제 
필요한 작업에 대해서 함수 형태로 작성 후 호출이 필요하다

In [2]:
from dotenv import load_dotenv, find_dotenv
# .env 를 환경변수로 설정
load_dotenv(find_dotenv())

True

In [3]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano",
    input="2월 20일 삼성전자의 주식 가격을 알려줘"
)

print(response.output_text)

확인해드리려면 연도와 어떤 가격 정보를 원하시는지가 필요해요.

- 연도: 예를 들어 2026년 2월 20일인가요, 아니면 다른 해인가요?
- 가격 정보: 종가를 원하시나요, 아니면 시가/고가/저가도 필요하시나요?

원하시면 제가 웹에서 실제 데이터를 찾아서 정확한 값을 알려드리겠습니다. 요청 형식을 예로 들면: "2026년 2월 20일의 삼성전자 종가"처럼 알려주시면 됩니다.


In [ ]:
# 야후 파이넨스 호출 -> 보편적으로사용됨
# !pip install yfinance

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 47.8 MB/s  0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15690 sha256=9e6c9e3f94a80fd490c02327c95f86931e0ba042ae82acee042ac35921606ff4
  Stored in directory: c:\users\soldesk\appdata\local\pip\cache\wheels\7e\62\f9\20d7dbb144b6f563edab8e3a7fda71d976870cd41972035cdd
Successfully built multitasking

   ---------------------------------------- 0/7 [peewee]
   ---------------------------------------- 0/7 [peewee]
   ---------------------------------------- 0/7 [peewee]
   ----- --------------------


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import yfinance as yf
# 삼성전자 종목 코드로 조회
stock = yf.Ticker("005930.KS")
# data = stock.history(start="2026-02-20")
# 1d, 5d, 1m etc...
data = stock.history(period="1d")
print(data)

                               Open      High       Low     Close   Volume  \
Date                                                                         
2026-02-23 00:00:00+09:00  194400.0  197600.0  194300.0  195300.0  9184570   

                           Dividends  Stock Splits  
Date                                                
2026-02-23 00:00:00+09:00        0.0           0.0  


In [16]:
# 1) 스키마 정의 : json 스키마
# 2) 함수 작성
import json

tools = [
    {
        "type" : "function",
        "name": "get_korea_stock_price",
        "description" : "한국 주식 종목코드를 받아 현재 가격을 조회합니다",
        "parameters": {
            "type" : "object",
            "properties":{
                "code":{
                    "type":"string",
                    "description":"6자리 한국 주식 코드(예:005930)"
                }
            },
            "required":["code"]
        }
    }
]

def get_korea_stock_price(code):
    symbol = f"{code}.KS"
    stock=yf.Ticker(symbol)
    data=stock.history(period="1d")
    if data.empty:
        return json.dumps({"error":"종목 코드를 확인해 주세요"})
    latest = data.iloc[-1]
    return json.dumps({
        "code":code,
        "current_price":float(latest['Close']),
        "open":int(latest['Open']),
        "high":float(latest['High']),
        "low":int(latest['Low']),
        "volume":int(latest['Volume']),
    })
def run_korea_stock_agent(prompt):
    # prompt 값에 따라 tools 사용여부 모델 판단
    response = client.responses.create(
        model="gpt-5-nano",
        tools=tools,
        input=[{"role":"user","content":prompt}]
    )

    # function 호출 결과만 수집
    function_calls = [item for item in response.output if item.type == "function_call"]
    if not function_calls:
        return response.output_text

    # 삼성전자 주가 알려줘
    # 삼성전자와 하이닉스 주가 알려줘 -> function calling 이 여러번 일 수 있음
    tool_outputs = []
    for call in function_calls:
        args = json.loads(call.arguments)
        result = get_korea_stock_price(**args)

        tool_outputs.append({
            "type":"function_call_output",
            "call_id":call.call_id, # 어떤 function_call 에 대한 결과인지 매핑
            "output":result
        })
    final_response = client.responses.create(
        model="gpt-5-nano",
        tools=tools,
        input=[
            {"role":"user","content":prompt},
            *response.output,*tool_outputs
        ]
    )
    return final_response

In [17]:
run_korea_stock_agent("005930 현재 주가 알려줘")

Response(id='resp_0aa58ee946d04b7200699bc178339081a3bd5c4501481cc59e', created_at=1771815288.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-nano-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_0aa58ee946d04b7200699bc178db0881a3862a20a7c38f29ed', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseOutputMessage(id='msg_0aa58ee946d04b7200699bc17b64b881a3a7ccba0d322c158a', content=[ResponseOutputText(annotations=[], text='005930(삼성전자) 현재가: 194,100원\n\n- 시가: 194,400원\n- 고가: 197,600원\n- 저가: 192,250원\n- 거래량: 12,253,315주\n\n필요하시면 실시간 업데이트로 다시 확인해 드릴게요.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_korea_stock_price', parameters={'type': 'object', 'properties': {'code': {'type': 'string', 'description': '6자리 한국 주식 코드(예:005930)'}}, 'required': ['code'], 'additio

In [26]:
from openai import OpenAI
import json
import requests

client = OpenAI()

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "도시/location 을 받아서 현재 날씨(기온, 풍속 등)을 조회합니다",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "예 : 'Seoul' 혹은 '서울' 같은 도시/지역명 입력",
                },
                "timezone": {
                    "type": "string",
                    "description": "예 : 'Asia/Seoul' 또는 생략 가능 ",
                    "default":"auto"
                },
            },
            "required": ["location"],
        },
    },
]

def get_weather(location,timezone):
    # 날씨 api 호출 : 위도 경도를 알아야만 되는 api임
    params = {"name":location,"count":1,"language":"ko"}
    geo = requests.get("https://geocoding-api.open-meteo.com/v1/search",params=params,timeout=30).json()
    results = geo.get("results")[0]
    if not results:
        return json.dumps({"error":f"{location} 위치를 찾지 못했습니다"})
    lat, lon = results.get('latitude'), results.get('longitude')
    print(f"위,경도 {lat},{lon}")


    url = "https://api.open-meteo.com/v1/forecast"

    weather_params = {
        "latitude":lat,
        "longitude":lon,
        "current_weather":"true",
        "timezone":timezone
    }
    weather_result = requests.get(url,params=weather_params,timeout=30).json()
    current_weather = weather_result.get('current_weather')
    if not current_weather:
        return json.dumps({"error":" 날씨 데이터를 가져오지 못했습니다"})
    # 받은 결과 모델에게 돌려주기 위한 json 구조 설정
    return json.dumps({
        "location":location,
        "latitute":lat,
        "longitude":lon,
        "temperature_c": current_weather.get("temperature"),
        "windspeed_kmh": current_weather.get("windspeed_kmh"),
        "winddirection_deg": current_weather.get("winddirection_deg"),
        "weathercode": current_weather.get("weathercode"),
        "time": current_weather.get("time"),

    })

def execute_tool(name,arguments):
    if name == "get_weather":
        return get_weather(**arguments)


def run_weather(prompt):
    # Create a running input list we will add to over time
    input_list = [
        {"role": "user", "content": prompt}
    ]
    # 1. 모델에게 tolls 을 사용할 수 있음을 명시
    response = client.responses.create(
        model="gpt-5-nano",
        tools=tools,
        input=input_list,
    )

    # function 호출 결과만 수집
    function_calls = [item for item in response.output if item.type == "function_call"]
    if not function_calls:
        return response.output_text


    tool_outputs = []
    for call in function_calls:
        args = json.loads(call.arguments)
        result = execute_tool(call.name,args)

        tool_outputs.append({
            "type":"function_call_output",
            "call_id":call.call_id, # 어떤 function_call 에 대한 결과인지 매핑
            "output":result
        })
    final_response = client.responses.create(
        model="gpt-5-nano",
        tools=tools,
        input=[
            {"role": "user", "content": prompt},
            *response.output,*tool_outputs
        ]
    )
    return final_response

In [27]:
print(run_weather("Seoul 날씨 어때?"))

위,경도 37.566,126.9784
Response(id='resp_02e4a4bf9cc2e3df00699bd246916481928125d5d7ab7a832e', created_at=1771819590.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-nano-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_02e4a4bf9cc2e3df00699bd246c0108192af92d1cc4220dddc', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseOutputMessage(id='msg_02e4a4bf9cc2e3df00699bd24c0c0881929f2027f8c9515946', content=[ResponseOutputText(annotations=[], text='지금 서울의 날씨는 대략 이렇게 보여요.\n\n- 현재 기온: 약 1.4°C\n- 하늘 상태: 맑음\n- 시간 기준: 현지 시각 13:00 (서울)\n- 풍속/바람 정보는 데이터가 제공되지 않아 확인 불가\n- 추가 정보 원하시면 습도, 강수 확률 등도 확인해 드릴 수 있어요\n- 옷차림 추천: 매우 추우므로 두꺼운 재킷이나 패딩, 모자/장갑도 챙기세요\n\n오늘이나 주간 예보도 필요하시면 말씀해 주세요.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_weather', parameters={'type': 'object', 'pro